In [1]:
# !pip install cv2
!pip3 install dlib
import os
import pickle
import numpy as np
import cv2
import face_recognition
import cvzone
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import storage
import numpy as np
from datetime import datetime

In [2]:
#UBICACIÓN DE LA DATABASE
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred, {
    'databaseURL':"https://faceattendacerealtime-bbc1f-default-rtdb.firebaseio.com/",
    'storageBucket':"faceattendacerealtime-bbc1f.appspot.com"
})

bucket = storage.bucket()

#ACTIVACIÓN DE LA CÁMARA PARA LAS CAPTURAS Y SU RESOLUCIÓN, CON 0 LA ABRIMOS, CON 1 LA CERRAMOS
#----------------------------------------------------------
#----------------------------------------------------------
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

#DECLARAMOS NUESTRA IMAGEN DE FONDO
#----------------------------------------------------------
imgBackground = cv2.imread('Resources/background.png')


#IMPORTANDO EL MODO DE IMÁGENES EN UNA LISTA
#----------------------------------------------------------
folderModePath = 'Resources/Modes'
modePathList = os.listdir(folderModePath)
imgModeList = []
for path in modePathList:
    imgModeList.append(cv2.imread(os.path.join(folderModePath,path)))

#print(len(imgModelList))

#CARGA DEL FICHERO CODIFICADO
print("Loading Encode File ...")
file = open("EncodeFile.p","rb")
encodeListKnownWithIds = pickle.load(file)
file.close()
encodeListKnown, studentIds = encodeListKnownWithIds
# print(studentIds)
print("Encode File Loaded")

#NOS VA A MOSTRAR LOS ESTADOS SEGÚN VAYA HACIENDO EL PROCESO
modeType = 0
counter = 0
id = -1
imgStudent = []
studentInfo=None
while True:
    sucess, img = cap.read()

#REDUCTION IMAGES SIZE TO QUICKLY PROCESS
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)

#SUPERPONEMOS EL FONDO CON SUS MEDIDAS 
#----------------------------------------------------------
    imgBackground[162:162 + 480, 55:55 + 640] = img 
#----------------------------------------------------------
#Y AÑADIMOS TAMBIÉN LAS OTRAS IMÁGENES QUE INSERTAREMOS PERO CON OTRAS MEDIDAS  
    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
    
#COMPARE IMAGES ONE TO ONE. 
    if faceCurFrame: 
        for encodeFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
            #print("matches", matches)
            #print("faceDis", faceDis)

            matchIndex = np.argmin(faceDis)
            # print("Match Index", matchIndex)

            if matches[matchIndex]:
                # print("Known Face Detected")
                # print(studentIds[matchIndex])

    #CREAMOS EL CUADRADO DELIMITADOR PARA CAPTURAR
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                bbox = 55 + x1, 162 + y1, x2 - x1, y2 - y1
                imgBackground = cvzone.cornerRect(imgBackground, bbox, rt=0)
                id = studentIds[matchIndex]                    
    #CREAMOS ESTE CONDICIONAL PARA COMPROBAR TODO EL PROCESO          
                if counter == 0:
                    cvzone.putTextRect(imgBackground, "Loading", (275, 400))
                    cv2.imshow("Face Attendance", imgBackground)
                    cv2.waitKey(1)
                    counter = 1
                    modeType = 1

        if counter != 0 :

            if counter == 1:
    #OBTENIENDO LOS DATOS            
                studentInfo = db.reference(f'Students/{id}').get()
                
                print(studentInfo)

    #OBTENIENDO LA IMAGEN DEL ALMACENAMIENTO        
                blob = bucket.get_blob(f'Images/{id}.png')
                array = np.frombuffer(blob.download_as_string(), np.uint8)
                imgStudent = cv2.imdecode(array, cv2.COLOR_BGRA2BGR)
    #ACTUALIZACIÓN DE DATOS DE ASISTENCIA      
                datetimeObject = datetime.strptime(studentInfo['last_attendance_time'],
                                                   "%Y-%m-%d %H:%M:%S")
                secondsElapsed = (datetime.now() - datetimeObject).total_seconds()
                print(secondsElapsed)
                if secondsElapsed > 30:
                    ref = db.reference(f'Students/{id}')
                    studentInfo['total_attendance'] += 1
                    ref.child('total_attendance').set(studentInfo['total_attendance'])
                    ref.child('last_attendance_time').set(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                else:
                    modeType = 3
                    counter = 0
                    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
 
            if modeType != 3:        

                if 10 < counter < 20:
                    modeType = 2
 
                imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
 
                if counter <= 10:
                    cv2.putText(imgBackground, str(studentInfo['total_attendance']), (861, 125),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(studentInfo['major']), (1006, 550),
                                cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(id), (1006, 493),
                                cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(studentInfo['standing']), (910, 625),
                                cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 1)
                    cv2.putText(imgBackground, str(studentInfo['year']), (1025, 625),
                                cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 1)
                    cv2.putText(imgBackground, str(studentInfo['starting_year']), (1125, 625),
                                cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 1)
 
                    (w, h), _ = cv2.getTextSize(studentInfo['name'], cv2.FONT_HERSHEY_COMPLEX, 1, 1)
                    offset = (414 - w) // 2
                    cv2.putText(imgBackground, str(studentInfo['name']), (808 + offset, 445),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (50, 50, 50), 1)
 
                    imgBackground[175:175 + 216, 909:909 + 216] = imgStudent
 
                counter += 1

                

                if counter >= 20:
                    counter = 0
                    modeType = 0
                    studentInfo = []
                    imgStudent = []
                    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
    else:
        modeType = 0
        counter = 0                
            
#SALE LA CÁMARA Y NUESTRO PHOTOCALL DE FONDO
#----------------------------------------------------------
    # cv2.imshow("Webcam", img)
    cv2.imshow("Face Attendance", imgBackground)
    if cv2.waitKey(1)==27:
        break

Loading Encode File ...
Encode File Loaded


QObject::moveToThread: Current thread (0x560d0505f400) is not the object's thread (0x560d05dcd7f0).
Cannot move to target thread (0x560d0505f400)

QObject::moveToThread: Current thread (0x560d0505f400) is not the object's thread (0x560d05dcd7f0).
Cannot move to target thread (0x560d0505f400)

QObject::moveToThread: Current thread (0x560d0505f400) is not the object's thread (0x560d05dcd7f0).
Cannot move to target thread (0x560d0505f400)

QObject::moveToThread: Current thread (0x560d0505f400) is not the object's thread (0x560d05dcd7f0).
Cannot move to target thread (0x560d0505f400)

QObject::moveToThread: Current thread (0x560d0505f400) is not the object's thread (0x560d05dcd7f0).
Cannot move to target thread (0x560d0505f400)

QObject::moveToThread: Current thread (0x560d0505f400) is not the object's thread (0x560d05dcd7f0).
Cannot move to target thread (0x560d0505f400)

QObject::moveToThread: Current thread (0x560d0505f400) is not the object's thread (0x560d05dcd7f0).
Cannot move to tar

{'last_attendance_time': '2023-02-06 16:52:51', 'major': 'Data scientist', 'name': 'Javier', 'standing': 'G', 'starting_year': 2017, 'total_attendance': 144, 'year': 4}
176.639323
{'last_attendance_time': '2023-02-06 16:55:47', 'major': 'Data scientist', 'name': 'Javier', 'standing': 'G', 'starting_year': 2017, 'total_attendance': 145, 'year': 4}
8.239958
{'last_attendance_time': '2023-02-06 16:55:47', 'major': 'Data scientist', 'name': 'Javier', 'standing': 'G', 'starting_year': 2017, 'total_attendance': 145, 'year': 4}
9.573313
{'last_attendance_time': '2023-02-06 16:55:47', 'major': 'Data scientist', 'name': 'Javier', 'standing': 'G', 'starting_year': 2017, 'total_attendance': 145, 'year': 4}
10.734249
{'last_attendance_time': '2023-02-06 16:55:47', 'major': 'Data scientist', 'name': 'Javier', 'standing': 'G', 'starting_year': 2017, 'total_attendance': 145, 'year': 4}
11.902927
{'last_attendance_time': '2023-02-06 16:55:47', 'major': 'Data scientist', 'name': 'Javier', 'standing': '